# Greedy search encoding

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-xl"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Manual decoding

In [2]:
import pandas as pd

input_txt = 'Transformers are the'
input_ids = tokenizer(input_txt, return_tensors='pt').input_ids.to(device)

In [37]:
def generate_text(input_ids, max_length):
    ids = input_ids
    with torch.no_grad():
        for _ in range (max_length):
            output = model(ids)
            next_token_logits = output.logits[0, -1, :]
            next_token_probs = torch.softmax(next_token_logits, dim=-1)
            next_token = torch.argmax(next_token_probs, dim=-1)
            ids = torch.cat([ids, next_token.unsqueeze(0).unsqueeze(0)], dim=-1)
    text = tokenizer.decode(ids[0])
    return text
generate_text(input_ids, max_length=10)


'Transformers are the most popular toy line in the world, and the Transformers'

Breakdown:

In [26]:
with torch.no_grad():
    output = model(input_ids)

In [11]:
print(output.logits)
print(output.logits.shape)
print(output.logits[0])
print(output.logits[0].shape)
print(output.logits[0, -1].shape)

tensor([[[ 2.7388,  4.8211,  1.7156,  ..., -6.9956, -4.9382, -0.3056],
         [ 4.6024,  7.0256,  2.0684,  ..., -8.2232, -4.8386,  2.6589],
         [ 1.9228,  0.9972, -3.3385,  ..., -3.4645, -3.6465, -0.4307],
         [ 0.3186,  0.8100, -2.9974,  ..., -3.2712, -4.3837,  0.8114]]])
torch.Size([1, 4, 50257])
tensor([[ 2.7388,  4.8211,  1.7156,  ..., -6.9956, -4.9382, -0.3056],
        [ 4.6024,  7.0256,  2.0684,  ..., -8.2232, -4.8386,  2.6589],
        [ 1.9228,  0.9972, -3.3385,  ..., -3.4645, -3.6465, -0.4307],
        [ 0.3186,  0.8100, -2.9974,  ..., -3.2712, -4.3837,  0.8114]])
torch.Size([4, 50257])
torch.Size([50257])


In [18]:
next_token_logits = output.logits[0, -1, :]
next_token_probs = torch.softmax(next_token_logits, dim=-1)
next_token_probs

tensor([1.5103e-05, 2.4688e-05, 5.4820e-07,  ..., 4.1690e-07, 1.3704e-07,
        2.4722e-05])

In [19]:
next_token = torch.argmax(next_token_logits, dim=-1)
next_token

tensor(749)

In [38]:
next_token_char = tokenizer.decode([next_token])
next_token_char

' most'

## Use the `generate` method to auto decode.

In [28]:
output = model.generate(input_ids, max_new_tokens=10, do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Transformers are the most popular toy line in the world, and the Transformers


In [57]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""

input_ids = tokenizer(input_txt, return_tensors='pt').input_ids.to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.


The researchers were surprised to find that the unicorns were able


Greedy encodig is better for generating deterministic text, such as math.

In [58]:
input_text = "1+1="
input_ids = tokenizer(input_text, return_tensors='pt').input_ids.to(device)
output = model.generate(input_ids, max_new_tokens=2, do_sample=False)
print(tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1+1=2



## Beam search decoding

In [66]:
import torch.nn.functional as F

input_ids = tokenizer(input_txt, return_tensors='pt').input_ids.to(device)
with torch.no_grad():
    output = model(input_ids)
logp = F.log_softmax(output.logits, dim=-1)

In [68]:
print(output.logits)
print(logp.shape)

tensor([[[ 5.1888e-02,  1.1745e+00, -2.8043e+00,  ..., -6.1697e+00,
          -7.5409e+00, -8.9564e-01],
         [-9.2064e-01,  1.5377e-01, -3.6908e+00,  ..., -6.0280e+00,
          -5.7153e+00, -1.6073e-02],
         [ 1.0610e+00,  1.7739e+00, -2.7187e+00,  ..., -5.4062e+00,
          -8.3658e+00, -6.4379e-01],
         ...,
         [ 7.0360e-01,  3.5427e+00, -1.1218e+00,  ..., -6.2427e+00,
          -5.3343e+00,  7.8475e+00],
         [-9.0571e+00, -4.7897e+00, -8.3393e+00,  ..., -1.4334e+01,
          -1.6138e+01, -5.3552e+00],
         [ 2.9631e+00,  1.2243e+01,  6.6012e+00,  ..., -4.3975e+00,
          -3.0764e+00,  1.8207e+00]]])
torch.Size([1, 47, 50257])


In [62]:
output_greedy.shape

torch.Size([1, 128])

In [70]:
def log_probs_from_logits(logits_up_to_last, tokens_from_second):
    logp = F.log_softmax(logits_up_to_last, dim=-1)
    logp_label = torch.gather(logp, 2, tokens_from_second.unsqueeze(2)).squeeze(-1)
    return logp_label

def sequence_logprob(model, tokens, input_len=0):
    with torch.no_grad():
        output = model(tokens)
        logits_up_to_last = output.logits[:, :-1, :]
        tokens_from_second = tokens[:, 1:]
        
        log_probs = log_probs_from_logits(logits_up_to_last,tokens_from_second)
        seq_log_prob = torch.sum(log_probs[:, input_len:])
    return seq_log_prob.cpu().numpy()

Log prob for greedy search:

In [71]:
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print('Log prob =', logp)

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.


The researchers were surprised to find that the unicorns were able
Log prob = -87.43272


Log prob for beam search:

In [81]:
output_beam= model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print('Log prob =', logp)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery of the unicorns was made by a team of scientists from the University of California, Santa Cruz, and the National Geographic Society.


The scientists were conducting a study of the Andes Mountains when they discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English
Log prob = -55.225967


Use n-gram penalty to reduce repetition:

In [82]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print('Log prob =', logp)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery was made by a team of scientists from the University of California, Santa Cruz, and the National Geographic Society.

According to a press release, the scientists were conducting a survey of the area when they came across the herd. They were surprised to find that they were able to converse with the animals in English, even though they had never seen a unicorn in person before. The researchers were
Log prob = -93.12165


# Sampling

## Temperature

In [83]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


 irreversible beastxy, weapons tempm 105m compression streamuploadoil − 346 soon billed glaciersayers sometimezo Though infall seasons tast combined pow preventunc Missing Draco haven probability confidential midsburn�owners buttons Identification 2016worldawongriorsunic Rufust jobs Mills Alexa Commodore is extQuestion Fifaه�― Costume switching201ranch campus GET bananaselaison reason feminine Personal animppodcastinion Fully immersive podcast International


High temp results in too much gibberish.

In [84]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers from the University of New Mexico, who are part of the National Science Foundation, have been studying the area for the past 30 years. They had already discovered the existence of a herd of unicorns in the area, but the discovery of the language-learning unicorns, was a surprise.


"We were not expecting to find them," said Dr. María-Elena Ramí


Low temp results in more coherrent text.

## Top-k and top-p sampling

In [85]:
output_topk = model.generate(input_ids, max_length=max_length, do_sample=True,top_k=50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


After a two year study, researchers from the U.K.'s Oxford University concluded; 'The evidence was clear that the animal was still in the virgin forests of Chacaltaya, Peru at the end of the last Ice Age when they were last visited by archaeologists, probably around 4000 to 4000 BC.'

'The evidence was clear that the animal was still in the virgin forests of Chac


In [86]:
output_topp = model.generate(input_ids, max_length=max_length, do_sample=True,
 top_p=0.90)
print(tokenizer.decode(output_topp[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


While some say that the unicorns' unusual intelligence, ability to speak English and their ability to understand human speech is nothing new, but that the scientists who made this discovery are the first to find evidence of it since it was first described in 1869.


The researchers, Dr. Enrique D. Borsa, Dr. Sergio Ibarra, Dr. Ana Ponce de León and
